In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 298 kB 5.0 MB/s 
     |████████████████████████████████| 3.1 MB 34.2 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 243 kB 48.5 MB/s 
     |████████████████████████████████| 1.1 MB 39.7 MB/s 
     |████████████████████████████████| 132 kB 46.8 MB/s 
     |████████████████████████████████| 3.3 MB 34.0 MB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 
     |████████████████████████████████| 271 kB 48.8 MB/s 
     |████████████████████████████████| 160 kB 48.6 MB/s 
     |████████████████████████████████| 192 kB 45.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=8f607fd3620dd422936072045217d1d34523bc04f699b5d698b2f3419c8c21c1
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72

In [ ]:
import transformers

print(transformers.__version__)

4.12.5


In [ ]:
task = "lid" # Should be one of "ner", "pos" or "chunk"
#model_checkpoint = "distilbert-base-uncased"
model_checkpoint = "xlm-roberta-base"
batch_size = 8

In [ ]:
tags_ind = ['Sinhala', 'English', 'Sin-Eng', 'Eng-Sin', 'Mixed', 'NameEntity', 'Symbol']

Summarize Dataset

In [ ]:
import pandas as pd

url = '/content/drive/MyDrive/FYP/Final/dataset/XLMR/withSemId/xlmr_Final_2.json'

tags_ind = ['Sinhala', 'English', 'Sin-Eng', 'Eng-Sin', 'Mixed', 'NameEntity', 'Symbol']

df = pd.read_json(url, lines=True)
count = 0
for labels in df['tags']:
  temp =[]
  for label in labels:
    temp.append(tags_ind.index(label))
  df['tags'][count] = temp
  count +=1

df

,tokens,tags
0,"[Ai, rate, neethiya, muta, balapannethida]","[2, 2, 2, 2, 2]"
1,"[hi, mobitel]","[1, 5]"
2,[Short],[1]
3,"[Wadakma, na, ,, signal, na, ,, ,, ,, mathugama]","[2, 2, 6, 1, 2, 6, 6, 6, 5]"
4,"[7812981, 13, සිගනල්, එන්නෙ, නැ]","[5, 5, 3, 0, 0]"
...,...,...
13416,"[Ha, zitizan, 2, mp, gadara]","[2, 1, 5, 5, 2]"
13417,"[Dialog, තරමි, වෙිගත්තව, නැ, අනිත්, එවා, ඒකයි,...","[5, 0, 0, 0, 0, 0, 0, 0, 0]"
13418,"[Cliq, plan, 3G, package, walata, wattamak, di...","[5, 1, 5, 1, 2, 2, 2, 2]"
13419,[Mata],[2]


Split train and test dataset

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#split the data into train and test set
#train,test = train_test_split(df, test_size=0.50, random_state=42)
trainData,testData = train_test_split(df, test_size=0.20, random_state=42)

#save the data
trainData.to_json('/content/drive/MyDrive/FYP/Final/dataset/XLMR/withSemId/train_sam_6.json', orient='records', lines=True,  force_ascii=False)
testData.to_json('/content/drive/MyDrive/FYP/Final/dataset/XLMR/withSemId/test_sam_6.json', orient='records', lines=True,  force_ascii=False)

Load Dataset

In [ ]:
from datasets import load_dataset, load_metric

dataset = load_dataset('json', data_files={'train': '/content/drive/MyDrive/FYP/Final/dataset/XLMR/withSemId/train_sam_6.json','test': '/content/drive/MyDrive/FYP/Final/dataset/XLMR/withSemId/test_sam_6.json'})
#dataset = load_dataset('json', data_files={'train': '/content/drive/MyDrive/FYP/Final/dataset/XLMR/train_sam.json','test': '/content/drive/MyDrive/FYP/Final/dataset/XLMR/test_sam.json'})
dataset

Using custom data configuration default-6ef0ae4a8e56bd33


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-6ef0ae4a8e56bd33/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 10736
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 2685
    })
})

Preprocessing Data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [ ]:
#import transformers
#assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'tags', 'tokens'],
        num_rows: 10736
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'tags', 'tokens'],
        num_rows: 2685
    })
})

Fine-tuning the model

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(tags_ind))

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [ ]:
#import torch

#device = "cuda" if torch.cuda.is_available() else "cpu"
#print(f"Device: {device}")
#if device == "cuda":
#    model = model.half()  # for half-precision training. reduces RAM requirement; decreases speed if on older GPU # https://huggingface.co/transformers/v1.1.0/examples.html
#model.to(device)
#model.train();

Device: cuda


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    #push_to_hub=True,
    #push_to_hub_model_id=f"{model_name}-finetuned-{task}",
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [tags_ind[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tags_ind[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
#grouped_params = model.parameters()
#optimizer=transformers.AdamW(grouped_params, lr=0.00025)
#scheduler=transformers.get_cosine_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=2000, num_training_steps=60000)
#optimizers = optimizer, scheduler

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    #optimizers=optimizers
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: tags, tokens.
***** Running training *****
  Num examples = 10736
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4026


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.184000,0.154704,0.953017,0.969628,0.961251,0.957784
2,0.121300,0.126065,0.966977,0.967398,0.967188,0.964799
3,0.087500,0.130764,0.968434,0.971014,0.969722,0.967196


Saving model checkpoint to test-lid/checkpoint-500
Configuration saved in test-lid/checkpoint-500/config.json
Model weights saved in test-lid/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-lid/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-lid/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-lid/checkpoint-1000
Configuration saved in test-lid/checkpoint-1000/config.json
Model weights saved in test-lid/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-lid/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-lid/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: tags, tokens.
***** Running Evaluation *****
  Num examples = 2685
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Sinhala s

TrainOutput(global_step=4026, training_loss=0.15321901456905845, metrics={'train_runtime': 1706.3855, 'train_samples_per_second': 18.875, 'train_steps_per_second': 2.359, 'total_flos': 626797597086144.0, 'train_loss': 0.15321901456905845, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: tags, tokens.
***** Running Evaluation *****
  Num examples = 2685
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Sinhala seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NameEntity seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Eng-Sin seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: English seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Symbol seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labe

{'epoch': 3.0,
 'eval_accuracy': 0.9671959257040144,
 'eval_f1': 0.9697224786835389,
 'eval_loss': 0.130764439702034,
 'eval_precision': 0.9684338983050848,
 'eval_recall': 0.9710144927536232,
 'eval_runtime': 22.3942,
 'eval_samples_per_second': 119.897,
 'eval_steps_per_second': 15.004}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [tags_ind[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [tags_ind[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: tags, tokens.
***** Running Prediction *****
  Num examples = 2685
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Sinhala seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NameEntity seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Eng-Sin seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: English seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Symbol seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labe

{'Eng': {'f1': 0.9717613836921991,
  'number': 8487,
  'precision': 0.9703912583715192,
  'recall': 0.9731353835277483},
 'Sin': {'f1': 0.9302514193025142,
  'number': 1227,
  'precision': 0.9257465698143664,
  'recall': 0.93480032599837},
 'ameEntity': {'f1': 0.8548839581739353,
  'number': 1958,
  'precision': 0.8537952114111055,
  'recall': 0.8559754851889684},
 'inhala': {'f1': 0.9835765379113018,
  'number': 8750,
  'precision': 0.9849856733524355,
  'recall': 0.9821714285714286},
 'ixed': {'f1': 0.25,
  'number': 25,
  'precision': 0.3333333333333333,
  'recall': 0.2},
 'nglish': {'f1': 0.9697666195190947,
  'number': 11269,
  'precision': 0.9660942316160281,
  'recall': 0.9734670334546099},
 'overall_accuracy': 0.9671959257040144,
 'overall_f1': 0.9697224786835389,
 'overall_precision': 0.9684338983050848,
 'overall_recall': 0.9710144927536232,
 'ymbol': {'f1': 0.9992101105845181,
  'number': 5061,
  'precision': 0.998618511940004,
  'recall': 0.9998024105907923}}

In [ ]:
model_path = '/content/drive/Shareddrives/FYP-CodeStars/Saved models/XLMR_LID'

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in /content/drive/Shareddrives/FYP-CodeStars/Saved models/XLMR_LID/config.json
Model weights saved in /content/drive/Shareddrives/FYP-CodeStars/Saved models/XLMR_LID/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/FYP-CodeStars/Saved models/XLMR_LID/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/FYP-CodeStars/Saved models/XLMR_LID/special_tokens_map.json


('/content/drive/Shareddrives/FYP-CodeStars/Saved models/XLMR_LID/tokenizer_config.json',
 '/content/drive/Shareddrives/FYP-CodeStars/Saved models/XLMR_LID/special_tokens_map.json',
 '/content/drive/Shareddrives/FYP-CodeStars/Saved models/XLMR_LID/tokenizer.json')

In [ ]:
from transformers import AutoModelForTokenClassification
model =  AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/FYP/Final/model")

In [ ]:
from transformers import AutoTokenizer
tokenizer =  AutoTokenizer.from_pretrained("/content/drive/MyDrive/FYP/Final/model")

In [ ]:
dataset['train'][0]['tokens']

['If', 'not', 'pay', 'my', 'money', 'back', 'plz', '.']

In [ ]:
from transformers import TokenClassificationPipeline

classifier = TokenClassificationPipeline(model=model, tokenizer=tokenizer, device=0)

classifier(dataset['train'][0]['tokens'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[[{'end': 2,
   'entity': 'LABEL_1',
   'index': 1,
   'score': 0.9974814,
   'start': 0,
   'word': '▁If'}],
 [{'end': 3,
   'entity': 'LABEL_1',
   'index': 1,
   'score': 0.9991883,
   'start': 0,
   'word': '▁not'}],
 [{'end': 3,
   'entity': 'LABEL_1',
   'index': 1,
   'score': 0.9917702,
   'start': 0,
   'word': '▁pay'}],
 [{'end': 2,
   'entity': 'LABEL_1',
   'index': 1,
   'score': 0.985394,
   'start': 0,
   'word': '▁my'}],
 [{'end': 5,
   'entity': 'LABEL_1',
   'index': 1,
   'score': 0.9973782,
   'start': 0,
   'word': '▁money'}],
 [{'end': 4,
   'entity': 'LABEL_1',
   'index': 1,
   'score': 0.99936414,
   'start': 0,
   'word': '▁back'}],
 [{'end': 3,
   'entity': 'LABEL_1',
   'index': 1,
   'score': 0.99764884,
   'start': 0,
   'word': '▁plz'}],
 [{'end': 1,
   'entity': 'LABEL_6',
   'index': 1,
   'score': 0.99988174,
   'start': 0,
   'word': '▁'},
  {'end': 1,
   'entity': 'LABEL_6',
   'index': 2,
   'score': 0.99988675,
   'start': 0,
   'word': '.'}]]